In [1]:
from utils.tf import *
from utils.db import *
from utils.gd2 import *
from utils.clt_repetidos import *
from datetime import datetime, time, timedelta
import cx_Oracle
import pandas as pd
import os
from utils.clt_repetidos import * 
from datetime import datetime
import polars as pl

In [2]:
df_clientes =  pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/ETL/11_04_2025/CLientesT_FINAL_ACTIVOS.csv")

In [ ]:
df_clientes["FECHA_NAC_O_CONST"] = df_clientes["FECHA_NAC_O_CONST"].apply(
    lambda x: (
        x if isinstance(x, str) and "NO SE TIENE REGISTRADA LA INFORMACION" in x
        else pd.to_datetime(str(x).split(",")[0].strip(), errors='coerce').date()
    ) # type: ignore
) # type: ignore

In [4]:
df_copy_clientes = df_clientes.copy()

In [71]:
rutas_dfs = {
    "MECA":'/home/ale1726/proyects/datalake/clientes/data/productos/MECA/data/21_04_2025/productos_clientes_meca.dat',
    "SIAG":'/home/ale1726/proyects/datalake/clientes/data/productos/SIAG/data/05_02_2025/productos_clientes_SIAG.dat',
    "SIMS":'/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/data/21_04_2025/productos_clientes_sims.dat',
    "SIPE":'/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/data/22_04_2025/productos_clientes_sipe.dat' ,
    "SIRAC":'/home/ale1726/proyects/datalake/clientes/data/productos/SIRAC/data/05_02_2025/productos_clientes_SIRAC.dat',
    "SOI": '/home/ale1726/proyects/datalake/clientes/data/productos/SOI/data/22_04_2025/productos_clientes_soi.dat',
    "TAS" : '/home/ale1726/proyects/datalake/clientes/data/productos/TAS/data/26_02_2025/productos_clientes_TAS_T.dat'
    }

name_busqueda =  ".+ TASA ACTIVO, S.A. DE C.V., F.I.R.V."
#name_busqueda = 'GARANTIAS CREDITO'

metadata = {
"MECA":{
    "ruta": '/home/ale1726/proyects/datalake/clientes/data/productos/MECA/data/21_04_2025/productos_clientes_meca.dat',
    "tabla": "",
    "col_id": "CLI_CLAVE",
    "col_transacciones": ["OPE_FOLIO","INSTRUMENTO","OPE_SER_NUM_SERIE","TIPO_MOVIMIENTO","OPE_NUM_TITULOS","OPE_IMPORTE_TITULOS","OPE_FECHA_ALTA"],
},      
"SIAG":{
    "ruta":'/home/ale1726/proyects/datalake/clientes/data/productos/SIAG/data/05_02_2025/productos_clientes_SIAG.dat',
    "tabla": "",
    "col_id": "NUMERO_CLIENTE",
    "col_transacciones": ['NUMERO_DE_GARANTIA', 'SUBPRODUCTO', 'FECHA_REGISTRO_GARANTIA', 'MONTO_CREDITO','PORCENTAJE_GARANTIZADO', 'MONTO_GARANTIZADO', 'TASA_INTERES','PORCENTAJE_COMISION'],
},
"SIMS":{
    "ruta": '/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/data/21_04_2025/productos_clientes_sims.dat',
    "tabla": "",
    "col_id": "CLAVE",
    "col_transacciones": ["FECHA_OPERACION","FOLIO", "MOVIMIENTO", "PORTAFOLIO", "VAR_MOVIMIENTO","MONTO_MOVIMIENTO","UTILIDAD","PERDIDA"],
},
"SIPE":{
    "ruta": '/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/data/22_04_2025/productos_clientes_sipe.dat' ,
    "tabla": "",
    "col_id": "BCO_CLAVE",
    "col_transacciones": ["IPE_FOLIO", "IPE_FECHA_REGISTRO", "IPE_IMPORTE", "IPE_PLAZO", "IPE_TASA", "IPE_FECHA_VENCIMIENTO"],
},
"SIRAC":{
    "ruta": '/home/ale1726/proyects/datalake/clientes/data/productos/SIRAC/data/05_02_2025/productos_clientes_SIRAC.dat',
    "tabla": "",
    "col_id": "CODIGO_CLIENTE",
    "col_transacciones":   [
            "NUMERO_PRESTAMO",
            "FECHA_APERTURA",
            "FECHA_VENCIMIENTO",
            "PRODUCTO",
            "DESC_LINEA_FINANCIERA",
            "MONTO_APROBADO",
            "MONTO_INICIAL",
            "TASA_TOTAL",
            "PLAZO",
            "NUMERO_CUOTAS",
            "VALOR_CUOTA",
        ],
},
"SOI":{
    "ruta": '/home/ale1726/proyects/datalake/clientes/data/productos/SOI/data/22_04_2025/productos_clientes_soi.dat',
    "tabla": "",
    "col_id": "BACVE_BANC",
    "col_transacciones": ["LINO_LINEA","LIF_FIRMA","LIMTO_LIN","TFDESC_CONT"],
},
"TAS" :{
    "ruta": '/home/ale1726/proyects/datalake/clientes/data/productos/TAS/data/26_02_2025/productos_clientes_TAS_T.dat',
    "tabla": "",
    "col_id": "NUMERO_CLIENTE",
    "col_transacciones": ["IORDEN","FOPER","FVENCE","IINSTR","NTINSTR", "TASA", "MONTO_REAL"],
},
}

In [8]:
df_copy_clientes["SOXID"] = df_copy_clientes.apply(
    lambda row: {f"{so}:{id}" for so, id in zip(row["SISTEMA_ORIGEN"].split(","), row["NUMERO_CLIENTE"].split(","))},
    axis=1
)

In [73]:
def convertir_si_es_numero(valor: str):
    try:
        return int(valor)
    except ValueError:
        try:
            return float(valor)
        except ValueError:
            return valor

def buscar_cliente(name_busqueda, metadata,  df_copy_clientes):
    info_trans_busqueda = {}

    filtro = df_copy_clientes["NOMBRE_O_RAZON_SOCIAL"].str.contains(name_busqueda, case=False, na=False)
    if not filtro.any():
        return {"error": f"No se encontró el nombre: {name_busqueda}"}

    try:
        lista_soxid = df_copy_clientes.loc[filtro, "SOXID"].iloc[0]
        resulta_ids = dict(soxid.split(":") for soxid in lista_soxid)

        for sistema, id_cliente in resulta_ids.items():
            ruta = metadata[sistema]["ruta"]
            if not ruta:
                continue

            id_cliente = convertir_si_es_numero(id_cliente)

            try:
                df_sis = pl.read_csv(ruta)
                filtrado = df_sis.filter(pl.col(metadata[sistema]["col_id"]) == id_cliente)
                filtrado = filtrado.fill_null("")
                if filtrado.is_empty():
                    continue
                info_trans_busqueda[sistema] = filtrado[metadata[sistema]["col_transacciones"]].to_dicts()
            except Exception as e:
                print(f"Error en {sistema}: {e}")
                continue

        return info_trans_busqueda

    except Exception as e:
        return {"error": str(e)}

In [74]:
transacciones = buscar_cliente("BANCO DE MEXICO", metadata, df_copy_clientes)

In [75]:
transacciones

{'error': "' MECA'"}

In [ ]:
df_clientes[df_clientes["SISTEMA_ORIGEN"] == 'MECA']

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
36,37,INF NO DISP,ABA DIVISAS CASA DE CAMBIO SA CV (GUAD),ABADIV,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
37,38,INF NO DISP,"ABA DIVISAS CASA DE CAMBIO, SA DE CV",ABA,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,3 26 80 00,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
100,100,UNIDAD DE MERCADO DE CAPITALES,ACCIONES Y VALORES DE MEXICO,ACCIV,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,3-26-48-48 JORGE BUSTOS,NO SE TIENE REGISTRADA LA INFORMACION,AVM760909UK0,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
137,137,UNIDAD DE MERCADO DE CAPITALES,ACTINVER CASA DE BOLSA,ACTINV,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,52-82-62-00 FAX: 52-82-62-24,NO SE TIENE REGISTRADA LA INFORMACION,ACB7609076M2,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
578,576,INF NO DISP,AMRO BANK MEXICO,AMRO,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12536,12417,INF NO DISP,"VECTODIVISAS CASA DE CAMBIO,SA CV (MTY)",VECTOM,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
12537,12418,INF NO DISP,"VECTODIVISAS CASA DE CAMBIO,SA DE CV",VECTO,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
12549,12430,INF NO DISP,VECTORMEX,VECTOR,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25
12563,12444,INF NO DISP,VER FCMX,FCHINY,VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,2-83-33-00,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,MECA,11/04/25


In [79]:
df =  pd.read_csv(rutas_dfs["SIPE"],  low_memory=False)

In [82]:
df

,BCO_CLAVE_FFON,BCO_CLAVE,BCO_NOMBRE,SUC_NUMERO,CIP_DIV_CLAVE,CIP_TES_CLAVE,CIP_SUC_NUMERO,IPE_FOLIO,IPE_FECHA_REGISTRO,IPE_FECHA_MODIF,...,IPE_TES_CLAVE,IPE_SUC_NUMERO,IPE_FECHA_VALOR,IPE_IMPORTE,IPE_PLAZO,IPE_TASA,IPE_FECHA_VENCIMIENTO,IPE_FOLIO_ORIGEN,IPE_FECHA_CONCERTACION,IPE_EVENTO
0,BOTKUS33,1,BANK OF TOKYO-MITSUBISHI,1.0,USD,MAT,1.0,69.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,1.0,1997-02-07 00:00:00,100000000.0,3.0,5.0000,1997-02-10 00:00:00,NaN,NaN,NaN
1,NaN,6,BLADEX,6.0,USD,MAT,6.0,70.0,1997-02-07 00:00:00,1997-02-14 00:00:00,...,MAT,6.0,1997-02-07 00:00:00,10000000.0,7.0,5.4375,1997-02-14 00:00:00,NaN,NaN,NaN
2,BNPAUS3N,2,"BNP-PARIBAS, N.Y.",2.0,USD,MAT,2.0,71.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,2.0,1997-02-07 00:00:00,300000000.0,3.0,5.1250,1997-02-10 00:00:00,NaN,NaN,NaN
3,NaN,4,BANK OF MONTREAL,4.0,USD,MAT,4.0,72.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,4.0,1997-02-07 00:00:00,250000000.0,3.0,5.1250,1997-02-10 00:00:00,NaN,NaN,NaN
4,NaN,3,DANSKE BANK,3.0,USD,MAT,3.0,73.0,1997-02-07 00:00:00,1997-02-10 00:00:00,...,MAT,3.0,1997-02-07 00:00:00,300000000.0,3.0,5.1250,1997-02-10 00:00:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67044,NaN,289,"CREDIT INDUSTRIEL ET COMMERCIAL,NY",289.0,USD,MAT,289.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67045,NaN,251,"HSBC BANK, USA",251.0,USD,GCM,251.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67046,NaN,12,BANCO ANAHUAC,12.0,USD,GCM,12.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67047,NaN,4,BANK OF MONTREAL,4.0,USD,MAN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df[df["BCO_CLAVE"] == 77]

,BCO_CLAVE_FFON,BCO_CLAVE,BCO_NOMBRE,SUC_NUMERO,CIP_DIV_CLAVE,CIP_TES_CLAVE,CIP_SUC_NUMERO,IPE_FOLIO,IPE_FECHA_REGISTRO,IPE_FECHA_MODIF,...,IPE_TES_CLAVE,IPE_SUC_NUMERO,IPE_FECHA_VALOR,IPE_IMPORTE,IPE_PLAZO,IPE_TASA,IPE_FECHA_VENCIMIENTO,IPE_FOLIO_ORIGEN,IPE_FECHA_CONCERTACION,IPE_EVENTO
434,NaN,77,"BANCA CREMI, S.A.",77.0,USD,MAT,77.0,515.0,1997-03-07 00:00:00,1997-04-09 00:00:00,...,MAT,77.0,1997-03-10 00:00:00,2000000.0,30.0,6.5,1997-04-09 00:00:00,NaN,NaN,NaN
836,NaN,77,"BANCA CREMI, S.A.",77.0,USD,MAT,77.0,992.0,1997-04-09 00:00:00,1997-06-09 00:00:00,...,MAT,77.0,1997-04-09 00:00:00,2000000.0,61.0,7.0,1997-06-09 00:00:00,515.0,NaN,NaN
66981,NaN,77,"BANCA CREMI, S.A.",77.0,USD,GCM,77.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/productos/TAS/data/26_02_2025/productos_clientes_TAS_T.dat")

/tmp/ipykernel_24838/550213403.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/productos/TAS/data/26_02_2025/productos_clientes_TAS_T.dat")


In [38]:
df.head(5)

,NUMERO_CLIENTE,ESPROVEED,TIPO_PERSONA,NOMBRE,NOMLARGO,RFC,NLINEA,ICONTRATO,IORDEN,FOPER,...,DIA_FVENCE,IINSTR,ITINSTR,NTINSTR,TASA,MONEDA,MONTO,MONTO_ASIGNADO,MONTO_REAL,FECHA_REGISTRO
0,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,1040153,1040153,430215,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,3.302570e+09,3.302570e+09,2020-11-12
1,51000,False,Cuentas Propias,GARANTIAS CREDITO,GARANTIAS CREDITO,NFI3406305T0,51000,51000,430214,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.297121e+09,3.297121e+09,3.297121e+09,2020-11-12
2,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,1040153,1040153,430100,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,0.000000e+00,3.308033e+09,2020-11-12
3,51000,False,Cuentas Propias,GARANTIAS CREDITO,GARANTIAS CREDITO,NFI3406305T0,51000,51000,430063,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,0.000000e+00,3.308033e+09,2020-11-12
4,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,1040153,1040153,329069,2020-04-01,...,25,DEPBANX2,OTROS,Otros Instrumentos,6.50,MXP,5.217592e+09,5.217592e+09,5.217592e+09,2020-04-01


In [29]:
df_filtrado = df_clientes[df_clientes['SISTEMA_ORIGEN'].str.contains(',')]

In [ ]:
df_filtrado

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
248,246,INF NO DISP,"AFIX SERVICIOS FINANCIEROS, S.A.P.I. DE C.V. S...","28563824, 1065491",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,36301888,,ASF0901169V3,SIN INFO,SIN INFO,HEREDIA HORNER HUGO,"PM, Moral Nacional no Gravable",INF NO DISP,"SIRAC, TAS",11/04/25
533,531,INF NO DISP,"ALSOL CONTIGO, S.A. DE C.V., SOFOM, E.N.R.","28627541, 1065418, 1065564",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,9676781434,,ACO090216551,SIN INFO,SIN INFO,GONZALEZ GARCIA DARIO -- GONZALEZ GARCIA JOSE ...,"PM, Moral Nacional Gravable",INF NO DISP,"SIRAC, TAS",11/04/25
811,809,INF NO DISP,APOYO ECONOMICO FAMILIAR SA DE CV SOFOM ENR,"22336018, 1064485",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,55 5229 0200,,AEF050221TY2,SIN INFO,SIN INFO,MESSMACHER HENRIQUEZ EDUARDO BERNHART -- OROCI...,"PM, Moral Nacional no Gravable",INF NO DISP,"SIRAC, TAS",11/04/25
1146,1140,INF NO DISP,"BANCA AFIRME, S.A.","306, 1041222",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,,,BAF950102JP5,SIN INFO,SIN INFO,ARZATE ORTIZ RAUL ROSENDO -- ASMITIA ANCONA JO...,"PM, Moral Nacional no Gravable",INF NO DISP,"TAS, SIPE",11/04/25
1150,1144,INF NO DISP,"BANCA CREMI,S.A.","BCDI, 77",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,"MECA, SIPE",11/04/25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11835,11727,INF NO DISP,"SOFIPA CORPORATION, S.A.P.I. DE C.V., SOFOM E....","28917282, 1065803",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,95 1501 7100,,FIN080623RZ8,SIN INFO,SIN INFO,GARCIA VERA GABRIEL -- HERNANDEZ DIAZ FRANCISCO,"PM, Moral Nacional no Gravable",INF NO DISP,"SIRAC, TAS",11/04/25
12378,12885,INF NO DISP,TRATON FINANCIAL SERVICES MEXICO S.A. DE C.V. ...,"4581019, 90066",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,MEXICO,...,52626674 -- 52626690 -- 5552628690 -- 55555555,NO SE TIENE REGISTRADA LA INFORMACION,NFI980108PX5,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,"PM, UNIONES DE CREDITO",INF NO DISP,"SIAG, SIRAC",11/04/25
12423,12311,INF NO DISP,UBS AG,"UBS, 5",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,"91382251 / 52, 800 813 9224",NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,"SIPE, SIMS",11/04/25
12510,12392,UNIDAD DE MERCADO DE CAPITALES,"VALORES MEXICANOS CASA DE BOLSA, S.A. DE C.V.","VALMEX, 1057519, 315",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,"566 6211, 5279 1200,",,"CBC79081421A, VMC930101JS6, XXX123456XXX",SIN INFO,SIN INFO,ALVAREZ TOLEDO CLAUDIA CECILIA -- PAYAN LOPEZ ...,"PM, Moral Nacional no Gravable",INF NO DISP,"MECA, TAS",11/04/25


In [14]:
df_filtrado["SOXID"] = df_filtrado.apply(
    lambda row: {f"{so}:{id}" for so, id in zip(row["SISTEMA_ORIGEN"].split(","), row["NUMERO_CLIENTE"].split(","))},
    axis=1
)


/tmp/ipykernel_24838/3449657443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["SOXID"] = df_filtrado.apply(


In [15]:
df_filtrado["SOXID"] 

248      {SIRAC:28563824,  TAS: 1065491}
533      { TAS: 1065418, SIRAC:28627541}
811      { TAS: 1064485, SIRAC:22336018}
1146           { SIPE: 1041222, TAS:306}
1150              {MECA:BCDI,  SIPE: 77}
                      ...               
11835    { TAS: 1065803, SIRAC:28917282}
12378      { SIRAC: 90066, SIAG:4581019}
12423               {SIPE:UBS,  SIMS: 5}
12510       {MECA:VALMEX,  TAS: 1057519}
12547         {MECA:VECT,  TAS: 1057520}
Name: SOXID, Length: 91, dtype: object